<h1>Image Classifier</h1>
Module Imports

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

<h2>Cuda Test</h2>
Code line used to check that cuda is running on the system

In [5]:
torch.randn(5).cuda()

tensor([-0.8394,  0.9832,  0.6699, -1.8981, -1.1155], device='cuda:0')

<h2>Neural Network Model Class</h2>
The class that defines the neural network.
Methods include the convolution layer made up of three blocks, each representing a layer in the network and params representing the number of nodes.
Conv_layer is used to filter the data and learn from it.
FC_layer is the feature configuration layer taking the filtered data to apply and recognise features.
Forward is the method to move a data, x, through the network to reach an output from input.

In [6]:
#flexible model
class CIFAR10Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        # Flatten images into vectors
         # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


model = CIFAR10Model().cuda()

<h2>Optimiser and Loss Functions</h2>
Using the inbuilt functionality from pytorch to create the loss and optimiser functions to train the neural network.

In [7]:
#define optimiser
params = model.parameters()
optimiser = optim.SGD(params, lr=1e-2)


In [8]:
#define loss
loss = nn.CrossEntropyLoss()

<h2>Load training and validation data</h2>
The CIFAR10 dataset is loaded here and transformed into tensors, whereby the 32x32 pixel images are transformed into multidimensional arrays and normalised so that it may be passed as input data to train the neural network. 

In [9]:
#train, val, label
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    ]
)

train_data = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)

val_size = 5000
batch_size = 64
train, val = random_split(train_data, [len(train_data)-val_size, val_size])
train_loader = DataLoader(train, batch_size=batch_size)
val_loader = DataLoader(val, batch_size=batch_size)

<h2>Training</h2>
This is where the model is trained. Epoch is the number of iterations the model trains for and outputs the loss value, how far the model was from the correct class, and accuracy of how many correct guesses it classified the images.
Training is split into two where the train_loader is used to adjust the network via gradient descent implemented by pyTorch, val_loader is used to test the model to ensure it is not overfitting to the training data where it runs the model without changing the network.

In [10]:
#training and validation loop

num_epochs = 8
for epoch in range(num_epochs):
    losses = list()
    accuracies = list()
    for batch in train_loader:
        x, y = batch
        x = x.cuda()
        y = y.cuda()
        l = model(x)
        J = loss(l, y)
        model.zero_grad()
        J.backward()
        optimiser.step()
        
        losses.append(J.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())
    print(f'Epoch {epoch + 1}, train loss: {torch.tensor(losses).mean():.2f}, train acc: {torch.tensor(accuracies).mean():.2f}')

    losses = list()
    accuracies = list()
    for batch in val_loader:
        x, y = batch
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            l = model(x)
        J = loss(l, y.cuda())
        
        losses.append(J.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())
    print(f'Epoch {epoch + 1}, val loss: {torch.tensor(losses).mean():.2f}, val acc: {torch.tensor(accuracies).mean():.2f}')
    

Epoch 1, train loss: 1.75, train acc: 0.35
Epoch 1, val loss: 1.50, val acc: 0.44
Epoch 2, train loss: 1.24, train acc: 0.54
Epoch 2, val loss: 1.22, val acc: 0.55
Epoch 3, train loss: 0.99, train acc: 0.64
Epoch 3, val loss: 1.27, val acc: 0.56
Epoch 4, train loss: 0.82, train acc: 0.71
Epoch 4, val loss: 1.08, val acc: 0.62
Epoch 5, train loss: 0.70, train acc: 0.75
Epoch 5, val loss: 0.89, val acc: 0.69
Epoch 6, train loss: 0.61, train acc: 0.78
Epoch 6, val loss: 0.76, val acc: 0.74
Epoch 7, train loss: 0.54, train acc: 0.81
Epoch 7, val loss: 0.79, val acc: 0.73
Epoch 8, train loss: 0.47, train acc: 0.84
Epoch 8, val loss: 0.69, val acc: 0.77


<h2>Save and Load</h2>
This is where the model is exported as a dictionary and then imported back to the script to ensure the model is saved.

In [11]:
#TODO: SAVE AND LOAD MODEL HERE; export as dict and load dict into nn module
PATH = './cifar_model.pth'
torch.save(model.state_dict(), PATH)
trained_model = "Load model into this variable?"

<h2>Load Test Data</h2>
Similar to the validation training, the test data, unseen data by the model, is loaded to test the correctness of the trained network.

In [12]:
test_dataset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size = len(test_dataset), shuffle=True)

<h2>Model Testing</h2>
This is where the test_loader is tested upon the trained model and output the accuracy of the model on 'real data' or data that it has not seen before how well was the generalisation of the training.

In [13]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        x, y = data
        x = x.cuda()
        y = y.cuda()
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y.cuda()).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 77 %
